# Import

In [1]:
import re
import json
import nltk
import numpy as np
import pandas as pd
from devon.devon import FSMStemmer

# Clean Texts

In [3]:
def _clean_by_length(text: str, min_len: int = 2, max_len: int = 30) -> str:
    word_list = text.split()
    result_list = []
    for word in word_list:
        if len(word) >= min_len and len(word) <= max_len:
            result_list.append(word)
    result_text = ' '.join(result_list)
    return result_text

def clean_text(text: str, stop_words: list) -> str:
    result_text_list = []
    splited_text = re.split(r"[^A-Za-z 'ʻʼ-]+", text)
    for sentence in splited_text:
        word_list = nltk.tokenize.WhitespaceTokenizer().tokenize(sentence)
        for word in word_list:
            if len(word) == 0 or word == '-':
                continue
            if word[0] == '-':
                word = word[1:]
            try:
                if word[-1] == '-':
                    word = word[:-1]
            except IndexError:
                pass
            lower_word = word.lower()
            stem_word = FSMStemmer().stem(words=lower_word)[0]
            if stem_word not in stop_words:
                result_text_list.append(stem_word)
    result_text = ' '.join(result_text_list)
    result_text = _clean_by_length(result_text)
    return result_text

In [4]:
dataframe = pd.read_csv("UzWikiTexts.csv")
stop_words = json.load(open("UzStopWords.json"))

In [5]:
lambda_ = lambda text: clean_text(text, stop_words)
dataframe['clean_text'] = dataframe.loc[:, 'sentence'].apply(lambda_)
dataframe = dataframe.dropna()

In [6]:
dataframe

,article_uuid,sentence,clean_text
0,2508e29b-6b60-42c5-bc8e-577f3a124cf1,ShoʻrvaShoʻrva — suyuq ovqat turi.,shoʻrvashoʻrva suyuq ovqat tur
1,2508e29b-6b60-42c5-bc8e-577f3a124cf1,"Tayyorlash usuliga koʻra, shoʻrvaning qovurma ...",tayyorlash usul koʻra shoʻrva qovurma shoʻrva ...
2,2508e29b-6b60-42c5-bc8e-577f3a124cf1,"Goʻsht boʻlaklarga boʻlinib, yogʻ bilan birga ...",goʻsht boʻlak boʻlinib yogʻ qizarguncha soʻngr...
3,2508e29b-6b60-42c5-bc8e-577f3a124cf1,"Suv qaynab chiqquncha qozonga tuz, bir dona qi...",suv qaynab chiqquncha qozon tuz dona qizil qal...
4,2508e29b-6b60-42c5-bc8e-577f3a124cf1,"Shoʻrva qaynab chiqqach, olovi pasaytirilib, y...",shoʻrva qaynab chiqqach olov pasaytirilib soat...
...,...,...,...
1157634,665ef546-82b2-4ac8-ab4d-07667cf39271,"Aholi zichligi – har kvadrat kilometrga 25,4 n...",ahol zichlig kvadrat kilometr kish geografiya ...
1157635,665ef546-82b2-4ac8-ab4d-07667cf39271,"Shundan 3,70 km2 quruqlik.",shun quruqlik
1157636,665ef546-82b2-4ac8-ab4d-07667cf39271,Dengiz sathidan oʻrtacha 290 m balandlikda joy...,de sath oʻrtacha balandlik joylashgan
1157637,de26d018-dbc2-4bc4-a55b-2567f60747bf,"Amenia, North Dakota",amenia north dakota


# Group data by page (pg_index)

In [7]:
def group_texts(dataframe: pd.DataFrame) -> pd.DataFrame:
    df_page_ind = pd.DataFrame(dataframe['article_uuid'].unique(), columns=['article_uuid'])
    df_page_ind = df_page_ind.reset_index().rename(columns={'index': 'pg_index'})
    dataframe = dataframe.merge(df_page_ind, how='left', on='article_uuid')
    dataframe = dataframe[['clean_text', 'pg_index']]
    dataframe = dataframe.groupby(by='pg_index').agg({'clean_text': ' '.join})
    df_pg = dataframe.reset_index()
    df_pg = df_pg.dropna()
    return df_pg

In [8]:
df_grouped_texts = group_texts(dataframe)

In [9]:
df_grouped_texts

,pg_index,clean_text
0,0,shoʻrvashoʻrva suyuq ovqat tur tayyorlash usul...
1,1,sho rva
2,2,sho rva
3,3,sho'rva
4,4,shoʻrkul suv omborishoʻrkul suv ombor buxoro v...
...,...,...
440968,440968,ambrose north dakota
440969,440969,ambrose shimoliy dakota
440970,440970,amenia shimoliy dakota amenia aqsh shimoliy da...
440971,440971,amenia north dakota


In [10]:
def create_textshape_data(dataframe: pd.DataFrame, text_shape: list, column: str) -> pd.DataFrame:
    data_text_shape = pd.DataFrame(columns=['text_shape', 'pages_amount'])
    for shape in text_shape:
        lambda_ = lambda text: len(text.split()) > shape
        pages_amount = dataframe[dataframe[column].apply(lambda_)].shape[0]
        data_text_shape = data_text_shape.append({'text_shape': shape,
                                                  'pages_amount': pages_amount}, ignore_index=True)
    return data_text_shape

In [11]:
create_textshape_data(df_grouped_texts, [100, 150, 300, 500, 1000], 'clean_text')

,text_shape,pages_amount
0,100,16214
1,150,9583
2,300,3495
3,500,1577
4,1000,581


Since we need to get a corpus of 10,000 documents, it is permissible that each document contains at least 150 words.

In [12]:
lambda_ = lambda text: len(text.split()) > 150
df_uz_corpus = df_grouped_texts[df_grouped_texts['clean_text'].apply(lambda_)].reset_index()

In [13]:
df_uz_corpus

,index,pg_index,clean_text
0,20,20,shoʻrlanishshoʻrlanish tuproq shoʻrlanish suv ...
1,62,62,shoʻro islomiya shoʻro islomiya islo kengash t...
2,102,102,shoʻrtan gaz-kimyo majmuasishoʻrtan gaz-kimyo ...
3,134,134,shoʻrtoʻgʻoyshoʻrtoʻgʻoy shoʻrtoʻqay jez davr ...
4,142,142,shoʻrxoklarshoʻrxok yuqor qatlam koʻproq suv o...
...,...,...,...
9578,437284,437284,favvorafavvora biror manba suv biror jo tushib...
9579,437305,437305,bj rkbj rk toʻliq is bj rk gu mundsd ttir tala...
9580,437889,437889,maks gorkiy noml umumiy oʻrta maktab maks gork...
9581,439612,439612,bayanavul tuman bayanavul tuma pavlodar viloya...


In [14]:
df_uz_corpus.to_csv("UzCleanCorpus.csv", index=False)